In [ ]:
# Smoke Test - Fabric Python Notebook Connection Testing
# ============================================================================
# Tests connections to:
# 1. Metadata SQL DB using notebookutils.data.connect_to_artifact()
# 2. Target DWH using Soda Core Fabric with Service Principal auth
# ============================================================================

# Smoke Test - Connection Validation

Quick test to validate both connections work before running full DQ checks.

In [ ]:
# Install soda-core-fabric
%pip install soda-core-fabric --quiet

In [ ]:
# Configuration
META_DB_NAME = "soda_db-3dbb8254-b235-48a7-b66b-6b321f471b52"
DWH_SERVER = "yndfhalt62tejhuwlqaqhskcgu-n3hvjhr6avluxog2ch3jdnb5ya.datawarehouse.fabric.microsoft.com"
DWH_DATABASE = "sample_dwh"

# Service Principal credentials
CLIENT_ID = "b9450ac1-a673-4e67-87de-1b3b94036a40"
CLIENT_SECRET = "<YOUR_CLIENT_SECRET>"

print("Configuration loaded")

In [ ]:
import notebookutils
from soda.scan import Scan

print("Imports successful")

## Test 1: Metadata DB Connection

Using `notebookutils.data.connect_to_artifact()` - should work without tokens.

In [ ]:
print("=" * 50)
print("TEST 1: Metadata DB (notebookutils.data)")
print("=" * 50)

try:
    conn = notebookutils.data.connect_to_artifact(
        META_DB_NAME,
        artifact_type="sqldatabase"
    )
    print("  Connection created!")

    # Test simple query
    df = conn.query("SELECT 'Hello from Metadata DB' AS message, DB_NAME() AS database_name")
    print("  Query executed!")
    print(df)
    print("  SUCCESS: Metadata DB connection works!")
except Exception as e:
    print(f"  FAILED: {e}")

## Test 2: Target DWH Connection (Soda Core)

Using Soda Core Fabric with Service Principal authentication.

In [ ]:
print("=" * 50)
print("TEST 2: Target DWH (Soda Core)")
print("=" * 50)

# Simple Soda check - just count rows in a table
soda_config = f"""
data_source fabric_dwh:
  type: fabric
  connection:
    driver: ODBC Driver 18 for SQL Server
    host: {DWH_SERVER}
    database: {DWH_DATABASE}
    authentication: activedirectoryserviceprincipal
    client_id: {CLIENT_ID}
    client_secret: {CLIENT_SECRET}
    encrypt: true
    trust_server_certificate: false
"""

# Simple check - just verify connection works
soda_checks = """
checks for INFORMATION_SCHEMA.TABLES:
  - row_count > 0:
      name: "Smoke test - tables exist"
"""

try:
    scan = Scan()
    scan.set_data_source_name("fabric_dwh")
    scan.set_scan_definition_name("smoke_test")
    scan.add_configuration_yaml_str(soda_config)
    scan.add_sodacl_yaml_str(soda_checks)

    print("  Executing Soda scan...")
    scan.execute()

    results = scan.get_scan_results()
    logs = scan.get_logs_text()

    if scan.has_error_logs():
        print("  FAILED with errors:")
        print(scan.get_error_logs_text())
    else:
        print("  SUCCESS: Soda scan completed!")
        print(f"  Results: {results}")

except Exception as e:
    print(f"  FAILED: {e}")

In [ ]:
print("=" * 50)
print("SMOKE TEST COMPLETE")
print("=" * 50)